<a href="https://colab.research.google.com/github/keppy/WorldEnder.ai/blob/master/WorldEnder_ai_Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Project Setup
import os
from pathlib import Path

# Download files on colab
if not Path("requirements.txt").exists():
    !wget https://raw.githubusercontent.com/keppy/WorldEnder.ai/master/requirements.txt
    !pip install -r requirements.txt -Uqq
if not Path("helpers.py").exists():
    !wget https://raw.githubusercontent.com/keppy/WorldEnder.ai/master/helpers.py

import openai
from getpass import getpass

# Setup your Openai API key
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

--2024-06-05 21:22:36--  https://raw.githubusercontent.com/keppy/WorldEnder.ai/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      70  --.-KB/s    in 0s      

2024-06-05 21:22:37 (1.32 MB/s) - ‘requirements.txt’ saved [70/70]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

# WordlEnder.ai
Presented by James Dominguez

## Can LLMs replace backends?

Let's look at some ways we as a community have grown.

## 1. The dark ages: un-typed prompting

In the begining, all we had was chatGPT and we would do something like the following if we wanted to get structured output:

In [ ]:
from openai import OpenAI

client = OpenAI()

resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Please give me character properties who were in the order of the phoenix as a json object ```json\n"},
    ],
    n=5,
    temperature=1,
)

for choice in resp.choices:
    json = choice.message.content
    try:
        person = json
        print(f"correctly parsed {person}")
    except Exception as e:
        print("error!!")
        print(json)

correctly parsed {
  "characters": [
    {
      "name": "Harry Potter",
      "house": "Gryffindor",
      "loyalty": "Dumbledore's Army",
      "role": "Leader"
    },
    {
      "name": "Hermione Granger",
      "house": "Gryffindor",
      "loyalty": "Dumbledore's Army",
      "role": "Member"
    },
    {
      "name": "Ron Weasley",
      "house": "Gryffindor",
      "loyalty": "Dumbledore's Army",
      "role": "Member"
    },
    {
      "name": "Albus Dumbledore",
      "house": "Gryffindor",
      "loyalty": "Order of the Phoenix",
      "role": "Leader"
    },
    {
      "name": "Severus Snape",
      "house": "Slytherin",
      "loyalty": "Order of the Phoenix",
      "role": "Double Agent"
    }
  ]
}
correctly parsed {
  "characters": [
    {
      "name": "Harry Potter",
      "age": 15,
      "house": "Gryffindor",
      "role": "Member of Dumbledore's Army"
    },
    {
      "name": "Hermione Granger",
      "age": 15,
      "house": "Gryffindor",
      "role": "Mem

You'll notice that the error rate surely has improved since the early days, however it's immediately obvious from a system design aspect that we have a few problems:
    


1.   The dictionary structure is random. We will have to have a lot of code to process and check these dictionaries and it will be brittle.
2.   We aren't sure what types we will get in the properties. Note that we sometimes see the string "Unknown" and sometimes the value `null` for a missing property.
3.  Ambiguity around what 'the order of the phoenix' is. Movie title, or fictional organization?

```can
    {
      "name": "Hermione Granger",
      "house": "Gryffindor",
      "loyalty": "Dumbledore's Army",
      "role": "Member"
    }
...
    {
      "name": "Molly Weasley",
      "age": "",
      "role": "Member of the Order of the Phoenix"
...
    {
      "name": "Severus Snape",
      "house": "Slytherin",
      "role": "Head of Slytherin House, member of the Order of the Phoenix",
      "animagus": false,
      "patronus": "doe"
    }
...
    {
      "name": "Neville Longbottom",
      "house": "Gryffindor",
      "role": "Member of Dumbledore's Army",
      "wand": "Cherry and unicorn hair"
    }
...
    {
      "name": "Sirius Black",
      "house": "Gryffindor",
      "role": "Member of the Order of the Phoenix",
      "wand": "Unknown",
      "patronus": "Unknown"
    }

```


## Finding Our Way; Pydantic

In an ironic twist of fate an old-man of a library from the Python landscape has been reborn as the cool-kid on the block. I have observed seasoned Python backend programers in the wild lamenting "why the sudden popularity of Pydantic?", a library that was widely used for tasks like input validation and application configuration declaration.

It turns out Pydantic grew into and powerful data validation library, and when PEP 593 introduced Annotated to Python in 2019, Pydantic took full advantage of the ability to attach runtime metadata to types without changing how type checkers interpret them.

In [ ]:
from pydantic import BaseModel

class Wizard(BaseModel):
    """
    A non-muggle character from the Harry Potter universe
    """
    name: str
    age: int
    house: str
    role: str

harry = Wizard(name="Harry Potter", age="15", house="Gryffindor", role="Dumbledore's Army")
print(harry)

name='Harry Potter' age=15 house='Gryffindor' role="Dumbledore's Army"


Note how when we pass in an argument that is the wrong type for `age`; Pydantic is smart enough to cast incorrect types to the correct type.

## Seeing The Light: Function Calling

There now exists a subset of large language models that are fine tuned to accept JSON schemas as input and return JSON as output. We can take advantage of this by using the inherent nature of our class. `BaseModel` gives us the ability to call `.model_json_schema()`; the OpenAI API lets us pass in the JSON schema description of a function, the model is trained to return appropriate arguments which **you** will call the function with.

In [ ]:
Wizard.model_json_schema()

{'description': 'A non-muggle character from the Harry Potter universe',
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'age': {'title': 'Age', 'type': 'integer'},
  'house': {'title': 'House', 'type': 'string'},
  'role': {'title': 'Role', 'type': 'string'}},
 'required': ['name', 'age', 'house', 'role'],
 'title': 'Wizard',
 'type': 'object'}

For our purposes it's important to note that the description is part of the JSON schema, and thus it will be used as input for the LLM. This is a cornerstone of WorldEnder.ai as we lean heavily on descriptions to guide the LLM towards generating content that fulfills the needs of our game logic and the user's unique narritive input.

Under the hood when we call the OpenAI API we define a `tools` array that holds our JSON schemas. Since functions are objects in Python we can pass our `Wizard` JSON schema and it will be valid as a `tool`--modern LLMs that are conforming to the OpenAI API can work with different tool modes read here for more in depth information: https://platform.openai.com/docs/guides/function-calling

## instructor

Instead of building up a tools array by hand with results from `.model_json_schema()` and validating results by hand with Pydantic, we can use a library called instructor which allows us to pass a new parameter to the AI client called `response_model`.*italicized text*

## Simulation Algorithm


1.  Generate an `Event` from a text player query that has a list of possible `Outcomes`
2.  The player sees *only* a list of 3-5 `choices` per `Outcome`–predicted to cause each `Outcome`
3.  The player selects one `choice`–the string is fed back into the `Event` generator (1.) as a new player query
4.  After *x* cycles of player multiple choice prompting, the final model outputs a `WorldEnder` prediction

